In [45]:
import os
import sys
sys.path.append(os.path.join(os.pardir, os.pardir))

import math
import random

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms.functional as F_transforms
import torchinfo

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from amlutils.task3 import UNet, load_zipped_pickle, visualize_segmentation

RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
pl.seed_everything(RANDOM_SEED, workers=True)

Epoch 70:   0%|          | 0/1 [11:38:50<?, ?it/s, loss=0.317, v_num=23]


Global seed set to 42


42

In [46]:
# CHANGE DIRECTORY
data_dir = os.path.join(os.pardir, 'data')

# Segment Image

In [ ]:
image = convert_to_tensor(train_set[0]['Image'])
image = torch.unsqueeze(image, 0)
segmentation = mv_segmenter(image)
segmentation = torch.squeeze(segmentation)
segmentation.shape

In [ ]:
prediction = F.softmax(segmentation, dim=0)
print(prediction.shape)
prediction = torch.max(prediction, dim=0).indices
prediction = torch.squeeze(prediction)#.detach().numpy()
#prediction = prediction[0,:,:].detach().numpy()
prediction

In [ ]:
prediction = F.softmax(segmentation, dim=0)
print(torch.min(prediction[0,:,:]))
prediction = torch.round(prediction[1,:,:]).detach().numpy()
np.max(prediction)

In [ ]:
prediction = F.softmax(segmentation, dim=0)
prediction = torch.where(prediction[1,:,:] > 0.29, torch.ones(prediction.shape[1:]), torch.zeros(prediction.shape[1:])).numpy()
prediction

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
fig.set_facecolor('white')

cropped_image = F_transforms.center_crop(image, prediction.shape[1:])
cropped_label = F_transforms.center_crop(torch.tensor(train_set[0]['Label']), prediction.shape[1:]).numpy()
cropped_image = torch.squeeze(cropped_image)
cropped_image = torch.squeeze(cropped_image).numpy()

visualize_segmentation(ax[0], cropped_image, prediction, segmentation_opacity=0.5)
ax[0].set_title('U-Net Predicted Segmentation (P[Is MV]>0.29)')

visualize_segmentation(ax[1], cropped_image, cropped_label, segmentation_opacity=0.5)
ax[1].set_title('Ground Truth')

plt.tight_layout()
plt.show()

# Inspect Training Set

In [ ]:
train_set = load_zipped_pickle(os.path.join(data_dir, 'labeled-images.pkl'))

In [ ]:
visualize_segmentation(train_set[0]['Image'], train_set[0]['Label'])

In [ ]:
train_input = torch.from_numpy(train_set[0]['Image'])
train_input = torch.unsqueeze(train_input, 0)
torchinfo.summary(UNet(), input_size=(1, *train_input.shape))

In [ ]:
visualize_segmentation(
    F_transforms.center_crop(torch.from_numpy(train_set[0]['Image']), prediction.shape).numpy(),
    F_transforms.center_crop(torch.from_numpy(train_set[0]['Label']), prediction.shape).numpy()
)

In [ ]:
tens = F_transforms.center_crop(torch.from_numpy(train_set[0]['Image']), (60, 60))
#torch.stack([tens, tens], dim=2).shape
tens = torch.unsqueeze(tens, dim=2)
torch.concat([tens, tens], dim=2).shape